In [1]:
from utils import *
import numpy as np
import pickle
# import torch

In [2]:
hindi_alphabet_to_index=pickle.load(open('vocab_tools/hindi_alphabet_to_index.pickle', 'rb'))
english_alphabet_to_index=pickle.load(open('vocab_tools/english_alphabet_to_index.pickle', 'rb'))
index_to_english_alphabet=pickle.load(open('vocab_tools/index_to_english_alphabet.pickle', 'rb'))
index_to_hindi_alphabet=pickle.load(open('vocab_tools/index_to_hindi_alphabet.pickle', 'rb'))

In [31]:
X_train=np.load('one_hot_data/X_train.npy')
X_valid=np.load('one_hot_data/X_val.npy')

y_train=np.load('one_hot_data/y_train.npy')
y_valid=np.load('one_hot_data/y_val.npy')

In [22]:
y_train.shape #index, seqlen, vocab/embedding length

(51199, 30, 68)

In [ ]:
class LSTM1(nn.Module): #src:-https://cnvrg.io/pytorch-lstm/ authored by Ahmad Anis
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length,dtype):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length
        self.dtype=dtype

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True).double() #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128).double() #fully connected 1
        self.fc = nn.Linear(128, num_classes).double() #fully connected last layer

        self.relu = nn.ReLU().double()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=my_device,dtype=self.dtype)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=my_device,dtype=self.dtype)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
import numpy as np
y_test=np.load('simple_data/y_train.npy')

In [13]:
y_test[0]

array([64,  0,  1,  2,  3,  4,  3,  5,  6, 65, 66, 66, 66, 66, 66, 66, 66,
       66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66])

In [26]:
import torch
from torch import nn
embedding = nn.Embedding(68,128)

In [27]:
torch.tensor(y_test[0])

tensor([64,  0,  1,  2,  3,  4,  3,  5,  6, 65, 66, 66, 66, 66, 66, 66, 66, 66,
        66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66])

In [36]:
y_valid[0].shape

(30, 68)

In [33]:
embedding(torch.tensor(y_valid[0])).shape

torch.Size([30, 68, 128])